In [134]:
'''Solving an Interger Linear Programming using Google OR-TOOLS with Python wrapper of GUROBI'''

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import networkx as nx

# import data
def rtv_data_prep(trip_size,fleet_size,seed_yes,disk_engine):
    T = pd.read_sql_query('SELECT *' 'FROM data' , disk_engine)
    T = T.loc[T['size']<=trip_size]
#import requests and vehicle locations
    man_veh = pd.read_csv('Manhattan_dropoff_14_80000-80030.csv',index_col='index')
    seed = [99999,22842,42034,73027,38470,23843,59173,38533,10238,83799]
    if seed_yes>0:
        seed_value = seed[seed_yes-1]
        man_veh_50 = man_veh.sample(frac=fleet_size,random_state=seed_value)
        T = T.loc[T['veh_index'].isin(man_veh_50.index)]
    return(T)


In [144]:
# Prepare data
def assign_data_prep(T):
    veh_rtv_filtered = T['veh_index'].unique().tolist()
    trip_rtv_filtered_str = T['request_index'].unique().tolist()
    trip_rtv_filtered = [int(x) for x in trip_rtv_filtered_str]
    unique_values_req =  pd.unique(T.loc[:,['r1','r2','r3','r4']].values.ravel()).tolist()
    unique_values_req = [int(x) for x in unique_values_req if str(x) != 'nan']
    unique_values_req.sort()
    num_vehicles = len(veh_rtv_filtered) #vehicle
    return(veh_rtv_filtered,trip_rtv_filtered_str,trip_rtv_filtered,unique_values_req,num_vehicles)

In [145]:
# Create graph of RTV-graph
def create_rtv_graph(unique_values_req,trip_rtv_filtered_str,T):
    RT_graph = nx.MultiGraph()
    for req in unique_values_req:
#     index_req = unique_values_req.index(req)
        RT_graph.add_node(req)
    for trip in trip_rtv_filtered_str:
#     index_trip = trip_rtv_filtered_str.index(trip)
        RT_graph.add_node(trip)
        trip_request_link = pd.unique(T.loc[T['request_index']==trip,['r1','r2','r3','r4']].values.ravel()).tolist()
        for trip_req in trip_request_link:
            if str(trip_req)!= 'nan':
#             index_req = unique_values_req.index(trip_req)
                RT_graph.add_edge(trip_req,trip)
    return(RT_graph)

In [146]:
# Create cost matrix (delay) of vehicle and trips
def create_cost_matrix(num_vehicles,trip_rtv_filtered_str,T,penalty_constant):
    cost = np.empty((0,num_vehicles+1))
    for trip in trip_rtv_filtered_str:
        cost_sub = []
        for vehicle in veh_rtv_filtered:
            vehicle_trips = T.loc[T['veh_index']==vehicle]
            if trip in vehicle_trips['request_index'].values:
                delay = vehicle_trips.loc[vehicle_trips['request_index']==trip].delay.values[0]
            else:
                delay = 0
            cost_sub.append(delay)
        #add trip cost of the dummy vehicle in the last column
        cost_sub.append(0)

    #     cost.append(cost_sub)
        cost = np.append(cost,[cost_sub],axis=0)

    #Add cost of rejecting request
    cost_reject = []
    for veh in veh_rtv_filtered:
        cost_reject.append(0)
    cost_reject.append(penalty_constant)
    for request in unique_values_req:
        cost = np.append(cost,[cost_reject],axis=0)
    return(cost)

In [149]:
# Initiate OR-TOOLS
from ortools.linear_solver import pywraplp
# Create the mip solver with gurobi
def create_solver():
    solver = pywraplp.Solver('SolveIntegerProblem',
                               pywraplp.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
    return(solver)

In [150]:
# Create cost matrix using variables of the solver
def create_x_matrix(trip_rtv_filtered,cost,solver):    
    count_obj = 0
    num_trips = len(trip_rtv_filtered) #trip
    x = np.empty((cost.shape[0],cost.shape[1]),dtype=object)
    for i in range(cost.shape[0]):
        for j in range(cost.shape[1]):
            if cost[i,j]>0:
                x[i,j] = solver.IntVar(0, 1, '')
            else:
                x[i,j] = 0
    return(x)

In [148]:
# Create request matrix that identify the combinations of requests that form each trip
def create_req_matrix(unique_values_req,num_trips,RT_graph,trip_rtv_filtered_str,num_vehicles,cost,x):    
    num_req = len(unique_values_req)
    r = np.empty((num_req,num_trips+num_req),dtype=object)
    for i in range(num_req):
        trip_index_all = []
        #get all edges connecting to a request and append to a list
        for edge in RT_graph.edges(unique_values_req[i]):
            trip_index = trip_rtv_filtered_str.index(edge[1])
            trip_index_all.append(trip_index)
        for j in range(num_trips):
            if j in trip_index_all:
                trip_req = []
                for n in range(num_vehicles+1):
                    if cost[j,n]>0:
                        trip_req.append(n)
                r[i, j] = sum([x[j, k] for k in trip_req])
            else:
                r[i, j] = 0
    for i in range(num_req):
        for j in range(num_req):
            if i==j:
                r[i,num_trips+j] = x[num_trips+i, num_vehicles] 
            else:
                r[i,num_trips+j] = 0
    return(r)

In [151]:
# Define the constraints
def define_constraints(x,num_req,num_trips,num_vehicles,penalty_constant,cost,unique_values_req,solver):
    # Each trip is assigned to at most 1 vehicle. (exclude last trip)
    # for i in range(num_trips):
    #     solver.Add(solver.Sum([x[i, j] for j in range(num_vehicles+1)]) <= 1)
    for i in range(x.shape[0]):
        solver.Add(solver.Sum([x[i, j] for j in range(x.shape[1])]) <= 1)

    # Each vehicle is assigned to at most one trip. (exclude last trip)
    for j in range(x.shape[1]-1):
        solver.Add(solver.Sum([x[i, j] for i in range(x.shape[0])]) <= 1)

    # Each request is assigned to exactly 1 trip (including rejected trip).
    for i in range(num_req):
        solver.Add(solver.Sum([r[i, j] for j in range(r.shape[1])]) == 1)

    objective_terms = []
    for i in range(num_trips):
        for j in range(num_vehicles):
            if cost[i,j]>0 and cost[i,j]<penalty_constant:
                objective_terms.append(cost[i,j] * x[i, j])
    #add cost of rejection 
    for i in range(len(unique_values_req)):
        objective_terms.append(cost[num_trips+i][num_vehicles] * x[num_trips+i,num_vehicles])
    return(objective_terms)

In [152]:
# Solve the interger programming
def solve_mip(objective_terms,solver):
    solver.Minimize(solver.Sum(objective_terms))
    status = solver.Solve()
    return(status)

In [153]:
def print_solution(seed_no,solver,Assignment_ILP):
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total Delay = ', solver.Objective().Value(),' Seed no.= ',seed_no, '\n')
        for i in range(num_trips + num_req):
            for j in range(num_vehicles):
                if not isinstance(x[i, j],int):
                    if x[i,j].solution_value() > 0.2:
#                         print('Trip %d assigned to vehicle %d.  Delay = %d' %
#                               (i, j, cost[i][j]))
                        Assignment_ILP = Assignment_ILP.append({'trip':trip_rtv_filtered_str[i],'vehicle':veh_rtv_filtered[j],'delay':cost[i][j],'solution_val':x[i, j].solution_value(),'seed_no':seed_no},ignore_index=True)
            if not isinstance(x[i, j+1],int) and x[i,j+1].solution_value() > 0.2:
#                 print('Trip %d assigned to vehicle %d.  Delay = %d' %
#                               (i, j+1, cost[i][j+1]))
                Assignment_ILP = Assignment_ILP.append({'trip':unique_values_req[i-num_trips],'vehicle':'rejected','delay':cost[i][j+1],'solution_val':x[i, j+1].solution_value(),'seed_no':seed_no},ignore_index=True)
    return(Assignment_ILP)

In [154]:
# combine all functions
def execute_all(trip_size,fleet_size,disk_engine):
    Assignment_ILP = pd.DataFrame()
    if fleet_size < 1:
        for i in range(10):
            T = rtv_data_prep(trip_size,fleet_size,i+1,disk_engine)
            veh_rtv_filtered,trip_rtv_filtered_str,trip_rtv_filtered,unique_values_req,num_vehicles = assign_data_prep(T)
            RT_graph = create_rtv_graph(unique_values_req,trip_rtv_filtered_str,T)
            cost = create_cost_matrix(num_vehicles,trip_rtv_filtered_str,T,penalty_constant)
            solver = create_solver()
            x = create_x_matrix(trip_rtv_filtered,cost,solver)
            r = create_req_matrix(unique_values_req,num_trips,RT_graph,trip_rtv_filtered_str,num_vehicles,cost,x)
            objective_terms = define_constraints(x,num_req,num_trips,num_vehicles,penalty_constant,cost,unique_values_req,solver)
            status = solve_mip(objective_terms,solver)
            print_solution(seed_no,solver,Assignment_ILP)
    return(Assignment_ILP)

In [ ]:
# Execute all functions
trip_size = 1 #1-4
fleet_size = 0.5 #0-1
disk_engine = create_engine('sqlite:///T_wait-5_delay-10_head20.db')
penalty_constant = 10000000
result_ilp_df = execute_all(trip_size,fleet_size,disk_engine)

In [100]:
Assignment_ILP.to_csv('ass_allveh_wait-5_delay-10_capa-1_veh-50-seed3.csv')